In [62]:
import spotipy 
import json
from spotipy.oauth2 import SpotifyClientCredentials
import config
import pandas as pd
import numpy as np
import pickle
import random
from sklearn import datasets 
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from matplotlib import pyplot
from sklearn.metrics import silhouette_score
from spellchecker import SpellChecker


spell = SpellChecker()



sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

my_music = pd.read_csv("my_music_clustered.csv", index_col=False)
top100 = pd.read_csv("top100songs.csv", index_col=False)



In [63]:
 with open("scaler.pickle", "rb") as f: 
            scaler = pickle.load(f) 
            
 with open("kmeans.pickle", "rb") as f: 
            kmeans = pickle.load(f) 

# Welcome to Song recommendation platform !

In [92]:
song_input = str(input("Please provide song name: "))

Please provide song name: What a Wonderful World


In [93]:
#check if spelling is correct, if not correct it!
for word in spell.unknown([song_input]):
   correct_word = spell.correction(word)

#check if song_input is in "hot chart"

if correct_word.lower() in list(top100["song_name"]):
    print("I recommend you this song: " + str(random.choice(top100["song_name"])))

else:
   
    results = sp.search(q=correct_word,limit=1,market="GB")
    song_id = results["tracks"]["items"][0]["id"]
    song_audio_features = sp.audio_features(song_id)[0]
    song_audio_features_df = pd.DataFrame(song_audio_features, index=[0])
    song_x = song_audio_features_df.select_dtypes(np.number)
    song_scaled = scaler.transform(song_x)
    kmeans.predict(song_scaled)
    output_features = my_music[my_music["cluster"] == kmeans.predict(song_scaled)[0]].sample()
    track_idd = output_features["id"].iat[0]




    
          

/Users/tuncaynasibli/opt/anaconda3/envs/DA_Env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(
/Users/tuncaynasibli/opt/anaconda3/envs/DA_Env/lib/python3.9/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KMeans was fitted with feature names
  warnings.warn(


In [94]:
from IPython.display import IFrame

#track_id = "1rfORa9iYmocEsnnZGMVC4"

IFrame(src="https://open.spotify.com/embed/track/"+track_idd,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )